In [80]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [81]:
empresas = ['AMER3', 'MGLU3', 'PETR4']

In [82]:
def getCsv(empresa):
    dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0', parse_dates=[['data', 'hora']], dayfirst=True)

    return dados.sort_values('data_hora')

In [83]:
amerdf = getCsv('AMER3')
amerdf.head()

,data_hora,url,titulo,texto
149,2023-06-06 05:00:00,https://br.investing.com/news/stock-market-new...,Americanas: ex-presidente Sergio Rial é o 1º a...,"['O executivo Sergio Rial, que renunciou à pre..."
148,2023-06-06 09:15:00,https://br.investing.com/news/stock-market-new...,Abertura de mercado: o que esperar do Ibovespa...,"['Por Jessica Bahia Melo', 'Investing.com – Os..."
147,2023-06-06 10:29:00,https://br.investing.com/news/stock-market-new...,"Ações: Locaweb, Petz e Méliuz registram maiore...","['Investing.com — O Ibovespa subia 0,49% às 10..."
146,2023-06-07 15:54:00,https://br.investing.com/news/stock-market-new...,Minoritários abrem queixa contra Americanas po...,"['Por Andre Romani e Carolina Pulice', 'SÃO PA..."
145,2023-06-09 08:41:00,https://br.investing.com/news/stock-market-new...,Site de acionistas que se sentiram lesados por...,['Um site que reúne acionistas que se sentiram...


In [84]:
amerdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 149 to 0
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   data_hora  150 non-null    datetime64[ns]
 1   url        150 non-null    object        
 2   titulo     150 non-null    object        
 3   texto      150 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.9+ KB


###################################################################################

In [85]:
def getDados(empresa, dataMax, dataMin):
    
    dados = yf.Ticker(empresa + '.SA')
    dados = dados.history(start=(dataMin - timedelta(days=2)), end=(dataMax + timedelta(days=2)), interval='1d')
    dados.reset_index(inplace=True)
    dados['data_hora'] = dados['Date']
    dados.drop(columns=['Date'], inplace=True)
    dados.set_index('data_hora', inplace=True)
    return dados

getDados('AMER3', amerdf['data_hora'].max(), amerdf['data_hora'].min()).index.max()

Timestamp('2023-09-21 00:00:00-0300', tz='America/Sao_Paulo')

In [86]:
def getDataMaisProxima(data):

    datas = {
                abs(data.timestamp() - date.timestamp()) : date for date in amer.index}
    
    dataMaisProxima = datas[min(datas.keys())]

    return dataMaisProxima - timedelta(days=1), dataMaisProxima, dataMaisProxima + timedelta(days=1), dataMaisProxima + timedelta(days=2)

In [95]:
def getValoresData(dados, data):
    try:
        valores = dados.iloc[dados.index.get_loc(data)]

    except KeyError:
        return {'Abertura' : None,
                'Fechamento' : None}

    return {'Abertura' : valores.Open,
            'Fechamento' : valores.Close}

In [96]:
def extraiValoresBolsa(empresa):
    df = getCsv(empresa)
    dados = getDados(empresa, df['data_hora'].max(), df['data_hora'].min())

    aberturaDiaAnterior = []
    aberturaDiaAtual = []
    aberturaDiaSeguinte = []
    abertura2Dias = []

    fechamentoDiaAnterior = []
    fechamentoDiaAtual = []
    fechamentoDiaSeguinte = []
    fechamento2Dias = []


    for data in df['data_hora']:
        diaAnterior, dataAtual, dataSeguinte, dia2 = getDataMaisProxima(data)

        valores = getValoresData(dados, diaAnterior)
        aberturaDiaAnterior.append(valores['Abertura'])
        fechamentoDiaAnterior.append(valores['Fechamento'])

        valores = getValoresData(dados, dataAtual)
        aberturaDiaAtual.append(valores['Abertura'])
        fechamentoDiaAtual.append(valores['Fechamento'])

        valores = getValoresData(dados, dataSeguinte)
        aberturaDiaSeguinte.append(valores['Abertura'])
        fechamentoDiaSeguinte.append(valores['Fechamento'])

        valores = getValoresData(dados, dia2)
        abertura2Dias.append(valores['Abertura'])
        fechamento2Dias.append(valores['Fechamento'])

    print('Aqui passou')

    df['aberturaDiaAnterior'] = aberturaDiaAnterior
    df['fechamentoDiaAnterior'] = fechamentoDiaAnterior

    df['aberturaDiaAtual'] = aberturaDiaAtual
    df['fechamentoDiaAtual'] = fechamentoDiaAtual

    df['aberturaDiaSeguinte'] = aberturaDiaSeguinte
    df['fechamentoDiaSeguinte'] = fechamentoDiaSeguinte

    df['abertura2Dias'] = abertura2Dias
    df['fechamento2Dias'] = fechamento2Dias

    return df

In [98]:
for empresa in empresas:
    print(empresa)
    df = extraiValoresBolsa(empresa)
    df.to_csv(empresa + '.csv')

AMER3
Aqui passou
MGLU3
Aqui passou
PETR4
Aqui passou
